In [4]:
import librosa
import librosa.display
import sklearn
import matplotlib.pyplot as plt

In [3]:
file = './Data/genres_original/classical/classical.00005.wav'
Signal , sr = librosa.load(file , sr = 22050)

In [ ]:
plt.figure(figsize=(15,5))
librosa.display.waveplot(Signal , sr = sr)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title("Classical music signal")
plt.show()